In [26]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tslearn.clustering import TimeSeriesKMeans
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, LSTM, Input, Flatten, Reshape
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
import numpy as np
import os
import pandas as pd
import pywt

In [27]:
epoch_num = 5
sequence_length = 6  # Number of time steps to consider

# Define file and directory names
file_name = '0_61938.csv'
data_dir = 'ohlc_data'
# parent_dir = 'forexPredictor'
# repo = 'Repos_git'
# repo_dir = 'Make_Money_with_Tensorflow_2.0'
# Get the absolute base directory dynamically
base_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Move up one level

# Construct the full file path in an OS-independent way
# data_path = os.path.join(base_dir, repo, repo_dir, parent_dir, data_dir, file_name)
data_path = os.path.join(base_dir, data_dir, file_name)

# Check if the file exists before using it
if os.path.exists(data_path):
    print(f"✅ File found: {data_path}")
else:
    print(f"❌ Error: File not found at {data_path}")


ucols=['Open', 'High', 'Low', 'Close']
data = pd.read_csv(data_path, usecols=ucols)
data.reset_index(drop=True, inplace=True)
data.head()


✅ File found: D:\Repos_git\Make_Money_with_Tensorflow_2.0\forexPredictor\ohlc_data\0_61938.csv


,Open,High,Low,Close
0,1.20997,1.21089,1.20966,1.20999
1,1.20481,1.20569,1.20479,1.20538
2,1.20537,1.20574,1.20341,1.20553
3,1.20556,1.20689,1.20442,1.20469
4,1.20468,1.20599,1.20380,1.20573


In [36]:
# Normalize data
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
data = scaler.fit_transform(df)

# Reshape for LSTM input
data = data.reshape((data.shape[0], data.shape[1], 1))

In [37]:
def apply_dwt(data):
    coeffs = [pywt.dwt(sample, 'haar') for sample in data]
    cA, cD = zip(*coeffs)  # Approximation and Detail coefficients
    return np.array(cA)  # Using only approximation coefficients

# Define Autoencoder with LSTM for better feature extraction
def create_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    x = LSTM(64, return_sequences=True)(input_layer)
    x = LSTM(32, return_sequences=False)(x)
    encoded = Dense(16, activation='relu')(x)

    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(input_shape[0], activation='sigmoid')(decoded)

    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder


In [29]:
df = data

In [30]:
# Convert DataFrame to NumPy array
data = df.values



In [38]:
# Apply DWT
transformed_data = apply_dwt(data)

# Train Autoencoder
autoencoder, encoder = create_autoencoder(transformed_data.shape[1:])
autoencoder.fit(transformed_data, transformed_data, epochs=5, batch_size=32, verbose=1)


Epoch 1/5
1936/1936 [==============================] - 9s 3ms/step - loss: 0.0089
Epoch 2/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 3/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 4/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059
Epoch 5/5
1936/1936 [==============================] - 6s 3ms/step - loss: 0.0059


In [ ]:
# Extract Features
features = encoder.predict(transformed_data)

# Apply TimeSeriesKMeans Clustering
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
tskmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="dtw", random_state=42)
clusters = tskmeans.fit_predict(features)


1936/1936 [==============================] - 3s 1ms/step


In [ ]:
# Convert clusters to one-hot encoding
clusters_one_hot = tf.keras.utils.to_categorical(clusters, num_clusters)

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, clusters_one_hot, test_size=0.2, random_state=42)

# Ensure input shape consistency
input_shape = features.shape[1:]  # Extract feature dimension
classifier_input = Input(shape=input_shape)
classifier_output = Dense(num_clusters, activation="softmax")(classifier_input)

# Create and compile classifier
final_model = Model(classifier_input, classifier_output)
final_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

epoch_num = 5  # Define the number of epochs
final_model.fit(X_train, y_train, epochs=epoch_num, batch_size=5, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


## Without DWT

In [34]:
# Define Autoencoder
def create_autoencoder(input_shape):
    input_layer = Input(shape=input_shape)
    x = Flatten()(input_layer)
    encoded = Dense(32, activation='relu')(x)
    encoded = Dense(16, activation='relu')(encoded)  # Ensure fixed-size encoding
    
    decoded = Dense(32, activation='relu')(encoded)
    decoded = Dense(np.prod(input_shape), activation='sigmoid')(decoded)
    decoded = Reshape(input_shape)(decoded)
    
    autoencoder = Model(input_layer, decoded)
    encoder = Model(input_layer, encoded)  # Encoder for feature extraction
    
    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Convert DataFrame to NumPy array
data = df.values

# Train Autoencoder
autoencoder, encoder = create_autoencoder(data.shape[1:])
autoencoder.fit(data, data, epochs=20, batch_size=32, verbose=1)

# Extract Features
features = encoder.predict(data)

# Apply TimeSeriesKMeans Clustering
num_clusters = 3  # Buy (2), Sell (1), Hold (0)
tskmeans = TimeSeriesKMeans(n_clusters=num_clusters, metric="euclidean", random_state=42)
clusters = tskmeans.fit_predict(features)

# Convert clusters to one-hot encoding
clusters_one_hot = tf.keras.utils.to_categorical(clusters, num_clusters)

# Split data into train and test sets (80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(features, clusters_one_hot, test_size=0.2, random_state=42)




Epoch 1/20
1936/1936 [==============================] - 3s 1ms/step - loss: 0.0230
Epoch 2/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 3/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 4/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 5/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 6/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 7/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 8/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 9/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 10/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 11/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
Epoch 12/20
1936/1936 [==============================] - 2s 1ms/step - loss: 0.0169
E

ValueError: in user code:

    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1360, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1349, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py", line 1126, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Teacher\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) fro

In [35]:
# Define classification model with correct input shape
classifier_input = Input(shape=(features.shape[1],))  # Use the encoded feature shape
classifier_output = Dense(num_clusters, activation="softmax")(classifier_input)

# Create and compile classifier model
final_model = Model(classifier_input, classifier_output)
final_model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the classifier
final_model.fit(X_train, y_train, epochs=epoch_num, batch_size=5, validation_data=(X_test, y_test))

# Evaluate model on test data
loss, accuracy = final_model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Epoch 1/20
9910/9910 [==============================] - 14s 1ms/step - loss: 1.0697 - accuracy: 0.4326 - val_loss: 1.0555 - val_accuracy: 0.4316
Epoch 2/20
9910/9910 [==============================] - 12s 1ms/step - loss: 1.0463 - accuracy: 0.4359 - val_loss: 1.0390 - val_accuracy: 0.4316
Epoch 3/20
9910/9910 [==============================] - 13s 1ms/step - loss: 1.0248 - accuracy: 0.4412 - val_loss: 1.0164 - val_accuracy: 0.4316
Epoch 4/20
9910/9910 [==============================] - 13s 1ms/step - loss: 1.0037 - accuracy: 0.4552 - val_loss: 0.9923 - val_accuracy: 0.4316
Epoch 5/20
9910/9910 [==============================] - 13s 1ms/step - loss: 0.9838 - accuracy: 0.4621 - val_loss: 0.9721 - val_accuracy: 0.4362
Epoch 6/20
9910/9910 [==============================] - 12s 1ms/step - loss: 0.9649 - accuracy: 0.4798 - val_loss: 0.9533 - val_accuracy: 0.4799
Epoch 7/20
9910/9910 [==============================] - 13s 1ms/step - loss: 0.9463 - accuracy: 0.5066 - val_loss: 0.9398 - val_ac

KeyboardInterrupt: 